In [ ]:
""" 
To plot results from anova plots.

HEre compare chan (coprrect trials) vs. chan (error trials).
"""

# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))


### LOAD MULTIPLE EXPERIMENTS FOR SUMMARY PLOT

In [15]:
from neuralmonkey.analyses.anova_agg_plots import load_and_preprocess_alldays, aggregate_df_var, plot_all, _extract_uses_rulecue2, _extract_list_event, _plot_get_eventsemantic_ordered
import seaborn as sns
from pythonlib.tools.snstools import rotateLabel

In [2]:
# LIST_DATE = [221020, 221112]
# LIST_DATE = [220814, 220815, 220816, 220827, 220913, 220921, 220928, 220929, 220930, 221001, 221014, 221020, 221021, 221031, 221102, 221107, 221112, 221114, 221119, 221121, 221125]

# Just good ones (eyeballed)
LIST_DATE = [220929, 220930, 221014, 221020, 221021, 221031, 221102, 221107, 221112, 221114, 221119, 221121, 221125]

# Just good ones, updated with a few extra (5/25/23)
LIST_DATE = [220929, 220930, 221001, 221002, 221014, 221020, 221021, 221023, 221024, 221031, 221102, 221107, 221112, 221113, 221114, 221118, 221119, 221121, 221125]

# LIST_DATE = [221020]
LIST_ANALY_VER = ["ruleswERROR", "rulesw"]
var_desired = "epoch"

LIST_DF_VAR = []
LIST_METADAT = []
LIST_PARAMS = []
for ANALY_VER in LIST_ANALY_VER:
#     ANALY_VER = "rulesw"
    score_ver = "r2_maxtime_1way_mshuff"
    animal = "Pancho"
    which_level = "trial"

    DF_VAR, METADAT, _PARAMS = load_and_preprocess_alldays(animal, ANALY_VER, var_desired, LIST_DATE, score_ver, which_level)

    DF_VAR["ANALY_VER"] = ANALY_VER
    LIST_DF_VAR.append(DF_VAR)
    LIST_METADAT.append(METADAT)
    LIST_PARAMS.append(_PARAMS)
    

DIR:  /gorilla1/analyses/recordings/main/anova/bytrial/MULT_DAY/Pancho-ruleswERROR-epoch-r2_maxtime_1way_mshuff-220929-221125
Loaded already-saved data!! /gorilla1/analyses/recordings/main/anova/bytrial/MULT_DAY/Pancho-ruleswERROR-epoch-r2_maxtime_1way_mshuff-220929-221125
Doing preprocess!!
0  ..  ['AnBmTR|0', 'AnBmTR|1', 'rndstr']  ..  ('AnBm', 'rankcol')
1  ..  ['AnBmTR', 'TR|0', 'TR|1', 'rndstr']  ..  ('AnBm', 'dir', 'rankcol')
2  ..  ['AnBmTR', 'TR', 'rndstr']  ..  ('AnBm', 'dir', 'rankcol')
3  ..  ['AnBmTR|0', 'AnBmTR|1', 'TR|0', 'TR|1', 'rndstr']  ..  ('AnBm', 'dir', 'rankcol')
4  ..  ['AnBmTR|0', 'AnBmTR|1', 'TR|0', 'TR|1', 'rndstr']  ..  ('AnBm', 'dir', 'rankcol')
5  ..  ['AnBmTR|0', 'AnBmTR|1', 'TR|0', 'TR|1', 'rndstr']  ..  ('AnBm', 'dir', 'rankcol')
6  ..  ['AnBmTR|0', 'AnBmTR|1', 'TR|0', 'TR|1', 'rndstr']  ..  ('AnBm', 'dir', 'rankcol')
7  ..  ['L', 'llV1']  ..  ('dir', 'shape')
8  ..  ['L', 'llV1']  ..  ('dir', 'shape')
9  ..  ['L', 'llV1']  ..  ('dir', 'shape')
10  ..  [

In [3]:
import pandas as pd
DF_VAR = pd.concat(LIST_DF_VAR, axis=0).reset_index(drop=True)


##### Clean the data

In [4]:
CLEAN = True
trial_block_kinds = ["blockfast", "trial"]
dates_exclude = [221001, 221002, 221021]
DF_VAR = DF_VAR[
    (DF_VAR["META_trial_or_block"].isin(trial_block_kinds)) & 
    ~(DF_VAR["META_date"].isin(trial_block_kinds))
]


In [ ]:
if False:
    from pythonlib.tools.pandastools import append_col_with_grp_index

    list_grouping = [
        ["META_trial_or_block", "is_same_beh_across_epochs"],
    #     ["epochs_kind", "META_trial_or_block", "is_same_beh_across_epochs"],
    ]

    # Plot
    dfthis = DF_VAR_AGG_EXPT.copy()
    for i, grouping in enumerate(list_grouping):
        dfthis = append_col_with_grp_index(dfthis, grouping, "_rowplot")

        for rowplot in dfthis['_rowplot'].unique():
            dfthisthis = dfthis[dfthis["_rowplot"]==rowplot]

            from pythonlib.tools.snstools import plotgood_lineplot
            # for val_kind in df_var_chan["val_kind"].unique():
            #     df_var_chan_this = df_var_chan[df_var_chan["val_kind"]==val_kind]
            fig = plotgood_lineplot(dfthisthis, xval="eventsemantic", yval="val", line_grouping="chan",
                                    include_mean=True, 
                                    relplot_kw={"col":"bregion"});
            #     rotateLabel(fig)
            #     for ax in fig.axes.flatten():
            #         ax.axhline(0, color="k")        
            #     fig.savefig(f"{savedir}/4_lineschans_feature_vs_region.pdf")


##### [ERROR TRIALS] Compare each channel to itself across analy vers

In [5]:
animal = _PARAMS["animal"]
analystring = "|".join(LIST_ANALY_VER)
SAVEDIR = f"/gorilla1/analyses/recordings/main/anova/bytrial/MULT_DAY_MULT_ANALY_VER/{animal}-{analystring}-{var_desired}-{score_ver}-{min(LIST_DATE)}-{max(LIST_DATE)}"
import os
os.makedirs(SAVEDIR, exist_ok=True)


In [ ]:

DF_VAR_AGG, DF_VAR_AGG_EXPT = aggregate_df_var(DF_VAR)


In [ ]:
from pythonlib.tools.pandastools import grouping_print_n_samples
if False:
    grouping_print_n_samples(DF_VAR, 
                             ["META_idx", "META_date", "META_trial_or_block", "is_same_beh_across_epochs", "var_others", "lev_in_var_others", "ANALY_VER"],
                             savepath=f"{SAVEDIR}/groupings.txt", save_as="txt", save_convert_keys_to_str=True)



In [6]:
### v1 - get each df, adn for each get diff of scores (for each chan).
from pythonlib.tools.pandastools import grouping_get_inner_items, grouping_append_and_return_inner_items, append_col_with_grp_index
def _dfvar_extract_matched(df_var, grouping, var_to_match, levels_to_get):
    """
    PARAMS;
    - grouping, each conjucntion must have each level in levels_to_get (of var_to_match)
    or else will throw out.
    """
    df_var = append_col_with_grp_index(df_var, grouping, "_grp")
    groupdict = grouping_get_inner_items(df_var, "_grp", var_to_match)

    list_df = []
    for grp, vals in groupdict.items():

        if sorted(vals) == sorted(levels_to_get):
            # Then good, has all analyver

            print("GOOD! .. ", grp, ' -- ', vals)

            dfthis = df_var[df_var["_grp"] == grp]
            list_df.append(dfthis)

    DF_VAR_MATCHED = pd.concat(list_df).reset_index(drop=True)
    
    return DF_VAR_MATCHED

grouping = ["META_date", "is_same_beh_across_epochs", "var_others", "lev_in_var_others", "eventsemantic"]
DF_VAR_MATCHED = _dfvar_extract_matched(DF_VAR, grouping, "ANALY_VER", LIST_ANALY_VER)

GOOD! ..  [221002, False, ('seqc_0_loc', 'seqc_0_shape', 'seqc_nstrokes_beh'), ((-1, 1), 'line-8-3-0', 4), 'baseline']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221002, False, ('seqc_0_loc', 'seqc_0_shape', 'seqc_nstrokes_beh'), ((-1, 1), 'line-8-3-0', 4), 'rulecueE']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221002, False, ('seqc_0_loc', 'seqc_0_shape', 'seqc_nstrokes_beh'), ((-1, 1), 'line-8-3-0', 4), 'rulecueL']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221002, False, ('seqc_0_loc', 'seqc_0_shape', 'seqc_nstrokes_beh'), ((-1, 1), 'line-8-3-0', 4), 'imageE']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221002, False, ('seqc_0_loc', 'seqc_0_shape', 'seqc_nstrokes_beh'), ((-1, 1), 'line-8-3-0', 4), 'imageL']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221002, False, ('seqc_0_loc', 'seqc_0_shape', 'seqc_nstrokes_beh'), ((-1, 1), 'line-8-3-0', 4), 'firstraise']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221002, False, ('seqc_0_loc', 'seqc_0_shape', 'seqc_nstrokes_beh'), ((-1, 1), 'line-8-3-0',

GOOD! ..  [221112, False, ('taskgroup', 'probe'), ('I', 0), 'rulecueE']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221112, False, ('taskgroup', 'probe'), ('I', 0), 'rulecueL']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221112, False, ('taskgroup', 'probe'), ('I', 0), 'imageE']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221112, False, ('taskgroup', 'probe'), ('I', 0), 'imageL']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221112, False, ('taskgroup', 'probe'), ('I', 0), 'firstraise']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221112, False, ('taskgroup', 'probe'), ('I', 0), 'stroke']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221112, False, ('taskgroup', 'probe'), ('I', 0), 'post']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221114, False, ('taskgroup', 'probe'), ('I', 0), 'rulecueL']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221114, False, ('taskgroup', 'probe'), ('I', 0), 'imageE']  --  ['ruleswERROR', 'rulesw']
GOOD! ..  [221114, False, ('taskgroup', 'probe'), ('I', 0), 'imageL']  --  ['rulesw

In [7]:
DF_VAR_MATCHED["grp"] = DF_VAR_MATCHED["_grp"] 

In [ ]:
# 2) much quicker, no need to pre-find cases that have both error and correct. This done
# autoamtically here.
from pythonlib.tools.pandastools import summarize_featurediff
dfsummary, dfsummaryflat, COLNAMES_NOABS, COLNAMES_ABS, COLNAMES_DIFF, dfpivot = summarize_featurediff(DF_VAR, GROUPING="ANALY_VER", GROUPING_LEVELS=["rulesw", "ruleswERROR"], 
                     FEATURE_NAMES=["val"], INDEX=["bregion", "chan", "eventsemantic", "META_date", "is_same_beh_across_epochs", "var_others", "lev_in_var_others"], return_dfpivot=True)
grouping_print_n_samples(dfsummary, grouping)

In [8]:
eventorder = _plot_get_eventsemantic_ordered(DF_VAR)

In [ ]:
if False:
    # plot
    %matplotlib inline
    import seaborn as sns
    from pythonlib.tools.snstools import rotateLabel
    fig = sns.catplot(data=dfsummary, x="eventsemantic", y="val-ruleswERRORminrulesw", col="bregion",
                      order = eventorder, row="date", kind="point", ci=68, aspect=1, hue="grp")
    rotateLabel(fig)
    fig.savefig(f"{sdir}/.pdf")



In [ ]:
# plot overlaid without taking difference

In [9]:
sdir = f"{SAVEDIR}/overlays"
os.makedirs(sdir, exist_ok=True)

In [10]:
import matplotlib.pyplot as plt


In [ ]:
if False: # TAKES TOO LOING
    import seaborn as sns
    list_date = DF_VAR_MATCHED["META_date"].unique()
    for date in list_date:

        dfthis = DF_VAR_MATCHED[DF_VAR_MATCHED["META_date"]==date]
        fig = sns.catplot(data=dfthis, x="eventsemantic", y="val", col="bregion",
                          hue="ANALY_VER", order = eventorder, kind="point", ci=68, 
                          row = "grp", aspect=1)

        rotateLabel(fig)
        for ax in fig.axes.flatten():
            ax.axhline(0, color="k", alpha=0.25)        
        fig.savefig(f"{sdir}/{date}.pdf")
        plt.close("all")


        ############# difference
        from pythonlib.tools.pandastools import summarize_featurediff
        dfsummary, dfsummaryflat, COLNAMES_NOABS, COLNAMES_ABS, COLNAMES_DIFF, dfpivot = summarize_featurediff(dfthis, GROUPING="ANALY_VER", GROUPING_LEVELS=["rulesw", "ruleswERROR"], 
                             FEATURE_NAMES=["val"], INDEX=["grp", "bregion", "chan", "eventsemantic", "META_date", "is_same_beh_across_epochs", "var_others", "lev_in_var_others"], return_dfpivot=True)
        fig = sns.catplot(data=dfsummary, x="eventsemantic", y="val-ruleswERRORminrulesw", col="bregion",
                      order = eventorder, row="grp", kind="point", ci=68, aspect=1)
        rotateLabel(fig)
        for ax in fig.axes.flatten():
            ax.axhline(0, color="k", alpha=0.25)
        fig.savefig(f"{sdir}/{date}_error_minus_rules.pdf")
        plt.close("all")


In [12]:
DF_VAR_MATCHED["bregion"]

0            M1_m
1            M1_m
2            M1_m
3            M1_m
4            M1_m
           ...   
90797    preSMA_a
90798    preSMA_a
90799    preSMA_a
90800    preSMA_a
90801    preSMA_a
Name: bregion, Length: 90802, dtype: object

In [16]:
# summary plot
dfthis = DF_VAR_MATCHED
fig = sns.catplot(data=dfthis, x="eventsemantic", y="val", col="bregion",
                  hue="ANALY_VER", order = eventorder, kind="point", ci=68, aspect=1)

rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.25)        
fig.savefig(f"{sdir}/ALL.pdf")
plt.close("all")

In [17]:
# Clean
CLEAN = True
bregions_in_order = ["M1_m", "PMd_p", "vlPFC_p", "vlPFC_a", "preSMA_a"]
dfthis = DF_VAR_MATCHED[
    (DF_VAR_MATCHED["bregion"].isin(bregions_in_order))
]

# summary plot
fig = sns.catplot(data=dfthis, x="eventsemantic", y="val", col="bregion",
                  hue="ANALY_VER", order = eventorder, kind="point", ci=68, aspect=1)

rotateLabel(fig)
for ax in fig.axes.flatten():
    ax.axhline(0, color="k", alpha=0.25)        
fig.savefig(f"{sdir}/ALL_subregions.pdf")
plt.close("all")

In [ ]:
if False:
    # NOT READY
    ############# difference
    from pythonlib.tools.pandastools import summarize_featurediff
    dfsummary, dfsummaryflat, COLNAMES_NOABS, COLNAMES_ABS, COLNAMES_DIFF, dfpivot = summarize_featurediff(dfthis, GROUPING="ANALY_VER", GROUPING_LEVELS=["rulesw", "ruleswERROR"], 
                         FEATURE_NAMES=["val"], INDEX=["grp", "bregion", "chan", "eventsemantic", "META_date", "is_same_beh_across_epochs", "var_others", "lev_in_var_others"], return_dfpivot=True)
    fig = sns.catplot(data=dfsummary, x="eventsemantic", y="val-ruleswERRORminrulesw", col="bregion",
                  order = eventorder, kind="point", ci=68, aspect=1)
    rotateLabel(fig)
    for ax in fig.axes.flatten():
        ax.axhline(0, color="k", alpha=0.25)
    fig.savefig(f"{sdir}/ALL_error_minus_rules.pdf")
    plt.close("all")

##### Testinmg

In [ ]:
DF_VAR

[autoreload of _distutils_hack failed: Traceback (most recent call last):
  File "/home/lucast4/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    except KeyError:
  File "/home/lucast4/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    old_objects.setdefault(key, []).append(weakref.ref(obj))
  File "/home/lucast4/miniconda3/envs/drag2_matlab/lib/python3.8/importlib/__init__.py", line 166, in reload
    spec = module.__spec__ = _bootstrap._find_spec(name, pkgpath, target)
  File "<frozen importlib._bootstrap>", line 914, in _find_spec
  File "/home/lucast4/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/_distutils_hack/__init__.py", line 95, in find_spec
    method_name = 'spec_for_{fullname}'.format(**locals())
NameError: name 'locals' is not defined
]
